In [1]:
import pandas as pd
import pyodbc
import useful_functions as use
from functools import reduce

In [3]:
npi_phones = pd.read_csv('../../Data/NPI/NPI_PHONES_TO_TRIANGULATE_041922.csv')
npi_phones['ME'] = use.fix_me(npi_phones.medical_education_number)
npi_phones['IQVIA_ME'] = [x[0:10] for x in npi_phones.ME]
npi_phones['PHONE_NPI'] = [use.fix_phone(x) for x in npi_phones.PHONE]
npi_phones.head()

,medical_education_number,PHONE,SOURCE,ME,IQVIA_ME,PHONE_NPI
0,102000323,4783014111,B,00102000323,0010200032,4783014111
1,102000439,2655355972,B,00102000439,0010200043,2655355972
2,102000510,8038029488,B,00102000510,0010200051,8038029488
3,102000561,4178310150,B,00102000561,0010200056,4178310150
4,102000676,3032864560,B,00102000676,0010200067,3032864560


In [5]:
#dhc
dhc = pd.read_csv('../../Data/DHC/DHC_2022-04.csv')
dhc = dhc[~dhc.ME.isna()] 
dhc = dhc.fillna('None')
dhc['ME'] = use.fix_me(dhc.ME)
dhc['DHC_PHONE'] = [use.fix_phone(x) for x in dhc['Phone Number']]
dhc = dhc[dhc.DHC_PHONE!='None']

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#iqvia
username = 'vigrose'
password = 'Hufflepuff~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT DISTINCT 
        B.PHONE,
        P.ME,
        T.AFFIL_TYPE_DESC,
        A.AFFIL_IND,
        A.AFFIL_RANK
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,PHONE,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK
0,2519493400,0010200001,Admitting,A,6
1,2564014000,0010200001,Admitting,P,9
2,7275885200,0010200001,Admitting,A,6
3,8282131111,0010200003,Admitting,A,6
4,8282131740,0010200003,Admitting (Inferred),P,9


In [7]:
iqvia =iqvia.fillna('None')
iqvia['IQVIA_PHONE'] = [use.fix_phone(x) for x in iqvia['PHONE']]
iqvia = iqvia[iqvia.IQVIA_PHONE!='None']

In [8]:
#symphony
sym_query = \
        """
        SELECT
        d.ADDR_FRST_TLPHN_NBR AS TELEPHONE,
        l.OTHER_ID AS SYM_ME
        FROM
        ODS.PRACTITIONER_DEMOGRAPHIC_LAYOUT d, ODS.PRACTITIONER_ADDL_IDS_LAYOUT l
        WHERE
        d.DS_PRCTR_ID = l.DS_PRCTR_ID
        and
        l.ID_QLFR_TYP_CDE = 38
        """  
symphony = pd.read_sql(con=ODS, sql=sym_query)
symphony.head()

,TELEPHONE,SYM_ME
0,(718) 756-1355,4954964003
1,(925) 779-5090,4954674001
2,(618) 235-2511,4954676001
3,(713) 864-6100,4954677001
4,(973) 754-2510,4954679001


In [11]:
symphony =symphony.fillna('None')
symphony['SYM_PHONE'] = [use.fix_phone(x) for x in symphony['TELEPHONE']]
symphony = symphony[symphony.SYM_PHONE!='None']

In [12]:
THIS = pd.merge(npi_phones, symphony, left_on='IQVIA_ME', right_on='SYM_ME', how='left')
THIS = pd.merge(THIS, iqvia, left_on='IQVIA_ME', right_on='ME', how='left', suffixes = ['','_iqvia'])
THIS = pd.merge(THIS, dhc, on='ME', how='left')[['ME','IQVIA_ME', 'PHONE_NPI', 'SYM_PHONE','IQVIA_PHONE', 'DHC_PHONE', 'SOURCE']].drop_duplicates()
THIS = THIS.fillna('None')
THIS.head()

,ME,IQVIA_ME,PHONE_NPI,SYM_PHONE,IQVIA_PHONE,DHC_PHONE,SOURCE
0,00102000323,0010200032,4783014111,2033890949,8046755000,None,B
1,00102000439,0010200043,2655355972,2565355972,2562651000,None,B
2,00102000510,0010200051,8038029488,8038029488,None,None,B
3,00102000561,0010200056,4178310150,4178310150,4172696000,4173348300,B
4,00102000561,0010200056,4178310150,4178310150,4172699800,4173348300,B


In [22]:
dict_list = []
for row in THIS.itertuples():
    count = 0
    MATCHES = ''
    phone = row.PHONE_NPI
    symph_phone = row.SYM_PHONE
    dhc_phone = row.DHC_PHONE
    iqvia_phone = row.IQVIA_PHONE
    if symph_phone == phone:
        count += 1
        MATCHES += 'Symphony, '
    if dhc_phone == phone:
        count += 1
        MATCHES += 'DHC, '
    if iqvia_phone == phone:
        count += 1
        MATCHES += 'IQVia, '
    if phone!='None':
        dicto = {
        'ME': row.ME,
        'IQVIA_ME': row.IQVIA_ME,
        'PHONE_NPI': phone,
        'MATCHED': count,
        'MATCHES': MATCHES[:-2],
        'SOURCE':row.SOURCE
    }
        dict_list.append(dicto)

In [23]:
phone_matches = pd.DataFrame(dict_list)

In [15]:
phone_matches = phone_matches.sort_values('MATCHED', ascending=False).drop_duplicates(['ME','PHONE_NPI'], keep='first')

In [28]:
testing = list(phone_matches[phone_matches.SOURCE!='B'].ME)

In [34]:
yup = phone_matches.sort_values('MATCHED', ascending=False).drop_duplicates(['ME','PHONE_NPI'], keep='first')

In [21]:
phone_matches.sort_values('MATCHED', ascending=False).drop_duplicates(['ME','PHONE_NPI'], keep='first')

,ME,IQVIA_ME,PHONE_NPI,MATCHED,MATCHES
77575,04113081169,0411308116,3523761611,3,"Symphony, DHC, IQVia"
96579,05107951514,0510795151,5086745600,3,"Symphony, DHC, IQVia"
103788,18745010021,1874501002,5207921450,3,"Symphony, DHC, IQVia"
49362,02879110257,0287911025,5139483600,3,"Symphony, DHC, IQVia"
63223,03575042511,0357504251,5083831000,3,"Symphony, DHC, IQVia"
...,...,...,...,...,...
56852,03508750501,0350875050,5162551600,0,
56849,03508750209,0350875020,8605459487,0,
56848,03508742141,0350874214,5106256262,0,
56845,03508741510,0350874151,9184942200,0,


In [41]:
yup.groupby('MATCHED').count()[['ME']]

,ME
MATCHED,
0,45754
1,21317
2,9340
3,445


In [40]:
yup.groupby('MATCHES').count()[['ME']]

,ME
MATCHES,
,45754
DHC,7267
"DHC, IQVia",359
IQVia,1117
Symphony,12933
"Symphony, DHC",8273
"Symphony, DHC, IQVia",445
"Symphony, IQVia",708


In [42]:
phone_matches[phone_matches.MATCHED!=0].sort_values('MATCHED', ascending=False).drop_duplicates(['ME','PHONE_NPI'], keep='first').to_excel('../../Data/NPI/NPI_Triangulation_Results_042022.xlsx', index=False)

In [39]:
12933/76856

0.16827573644217758

In [43]:
symphony_count = 0
dhc_count = 0
iqvia_count = 0
for row in THIS.itertuples():
    phone = row.PHONE_NPI
    symph_phone = row.SYM_PHONE
    dhc_phone = row.DHC_PHONE
    iqvia_phone = row.IQVIA_PHONE
    if symph_phone == phone:
        symphony_count += 1
    if dhc_phone == phone:
        dhc_count += 1
    if iqvia_phone == phone:
        iqvia_count += 1

In [46]:
dhc_count

28645

In [49]:
sample = phone_matches[phone_matches.MATCHED!=0].sort_values('MATCHED', ascending=False).drop_duplicates(['ME','PHONE_NPI'], keep='first')

In [52]:
sample = sample[['ME','PHONE_NPI']]

In [54]:
sample = sample.rename(columns={'PHONE_NPI':'phone_number'})

In [59]:
sample['FAX']=None

In [61]:
sample['SOURCE']='NPI'

In [62]:
sample['VERIFIED_DATE']='04/20/2022'

In [63]:
sample['LOAD_TYPE']=''

In [64]:
sample['FAX']=''

In [66]:
sample.to_csv('../../Data/NPI/NPI_VERIFIED_PHONE_LOAD_042022.csv', index=False)